In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
connection_string = "admin22:12345@localhost:5432/saccrewproject2"
engine = create_engine(f'postgresql://{connection_string}')

In [2]:
# Import Walmart 2019 Data
Walmart2019_df = "marketing_sample_for_walmart_com-ecommerce__20191201_20191231__30k_data.csv"
Walmart2019_df = pd.read_csv(Walmart2019_df)
Walmart2019_df = Walmart2019_df.rename(columns = {'List Price':'List Price 2019','Sale Price':'Sale Price 2019'}) #rename columns


In [3]:
#Create dataframe with only desired columns
Walmart2019_clean_df = Walmart2019_df[["Item Number","Product Name","Product Url","List Price 2019","Sale Price 2019","Brand","Category"]]


In [4]:
#Check if any columns have NaN values 
Walmart2019_clean_df.isna().any()

Item Number         True
Product Name       False
Product Url        False
List Price 2019    False
Sale Price 2019    False
Brand               True
Category            True
dtype: bool

In [5]:
#Drop the rows with NaN values
Walmart2019_clean_df = Walmart2019_clean_df.dropna()

In [6]:
# Import Walmart 2020 Data
Walmart2020_df = "marketing_sample_for_walmart_com-product_details__20200101_20200331__30k_data.csv"
Walmart2020_df = pd.read_csv(Walmart2020_df)
Walmart2020_df = Walmart2020_df.rename(columns = {'List Price':'List Price 2020','Sale Price':'Sale Price 2020'}) #rename columns


In [7]:
#Create dataframe with only desired columns
Walmart2020_clean_df = Walmart2020_df[["List Price 2020","Sale Price 2020","Item Number"]]


In [8]:
#Check if any columns have NaN values 
Walmart2020_clean_df.isna().any()

List Price 2020    False
Sale Price 2020    False
Item Number         True
dtype: bool

In [9]:
#Drop the rows with NaN values
Walmart2020_clean_df = Walmart2020_clean_df.dropna()


In [10]:
# Merge Item
Walmart_merged_df = pd.merge(Walmart2019_clean_df, Walmart2020_clean_df, on=['Item Number'], how="inner")

In [11]:
Walmart_merged_df2 = pd.DataFrame(Walmart_merged_df.Category.str.split('|',1).tolist(), columns = ['category1','category2'])


Walmart_merged_df2.head()

,category1,category2
0,Sports & Outdoors,Outdoor Sports | Hunting | Hunting Clothing |...
1,Food,"Condiments, Sauces & Spices | Chutneys & Reli..."
2,Sports & Outdoors,Outdoor Sports | Hunting | Game Cameras | All...
3,Pets,Pet Stain & Odor Remover
4,Sports & Outdoors,Sports | Tennis & Racquets | Tennis Racquets


In [12]:
Walmart_merged_df = pd.merge(Walmart_merged_df,Walmart_merged_df2, left_index=True, right_index=True)
Walmart_merged_df

,Item Number,Product Name,Product Url,List Price 2019,Sale Price 2019,Brand,Category,List Price 2020,Sale Price 2020,category1,category2
0,554298081.0,Realtree Ladies Softshell Jacket,https://www.walmart.com/ip/Realtree-Ladies-Sof...,75.22,75.22,Mossy Oak,Sports & Outdoors | Outdoor Sports | Hunting |...,91.94,91.94,Sports & Outdoors,Outdoor Sports | Hunting | Hunting Clothing |...
1,563412534.0,"Sechlers Hungarian Sweet-Mild Pepper Relish, 1...",https://www.walmart.com/ip/Sechler-s-Hungarian...,30.11,30.11,Sechlers,"Food | Condiments, Sauces & Spices | Chutneys ...",29.92,29.92,Food,"Condiments, Sauces & Spices | Chutneys & Reli..."
2,554946697.0,Stealth Cam STCG34 G Series Trail Camera 12 MP...,https://www.walmart.com/ip/Stealth-Cam-STCG34-...,101.59,101.59,Stealth Cam,Sports & Outdoors | Outdoor Sports | Hunting |...,106.42,106.42,Sports & Outdoors,Outdoor Sports | Hunting | Game Cameras | All...
3,555023216.0,"Odor Eaters Stink Stoppers for Kids and Teens,...",https://www.walmart.com/ip/Odor-Eaters-Stink-S...,4.23,4.23,Odor-Eaters,Pets | Pet Stain & Odor Remover,4.23,4.23,Pets,Pet Stain & Odor Remover
4,560814213.0,Head Jr Alum Tennis Rckt 21In,https://www.walmart.com/ip/Head-Jr-Alum-Tennis...,18.55,13.55,Generic,Sports & Outdoors | Sports | Tennis & Racquets...,18.55,18.55,Sports & Outdoors,Sports | Tennis & Racquets | Tennis Racquets
...,...,...,...,...,...,...,...,...,...,...,...
419,550820800.0,"061012 Whistle Lanyard, Single, Red",https://www.walmart.com/ip/061012-Whistle-Lany...,11.99,11.99,Mendota,Sports & Outdoors | Sports | Referee & Scoreke...,11.99,11.99,Sports & Outdoors,Sports | Referee & Scorekeeping
420,564332369.0,TRENDstarter Matte Finishing Wax,https://www.walmart.com/ip/TRENDstarter-Matte-...,9.63,9.63,TRENDstarter,Beauty | Beauty Next Day,9.63,9.63,Beauty,Beauty Next Day
421,574378945.0,Venum Undisputed 2.0 MMA Gloves,https://www.walmart.com/ip/Venum-Undisputed-2-...,62.99,62.99,Venum,Sports & Outdoors | Exercise & Fitness | Marti...,62.99,62.99,Sports & Outdoors,"Exercise & Fitness | Martial Arts | MMA, Mixe..."
422,551857126.0,"Barska 1"" High Dovetail-Style Airgun .22 Rings",https://www.walmart.com/ip/Barska-1-High-Dovet...,7.35,7.35,Barska,Sports & Outdoors | Outdoor Sports | Hunting |...,9.00,9.00,Sports & Outdoors,Outdoor Sports | Hunting | Optics & Binocular...


In [13]:
Walmart_merged_clean_df = Walmart_merged_df[["Item Number","Product Name","Product Url","List Price 2019","List Price 2020","Brand","category1"]]

In [14]:
Walmart_merged_clean_df = Walmart_merged_clean_df.rename(columns = {
    'List Price 2019':'price_2019',
    'List Price 2020':'price_2020',
    'Item Number': 'item_number',
    'Product Name': 'product_name',
    'Product Url': 'product_url',
    'Brand': 'brand',
    'category1': 'category'
})


In [15]:
Walmart_merged_clean_df = Walmart_merged_clean_df.loc[((Walmart_merged_clean_df['price_2019']>0) & (Walmart_merged_clean_df['price_2020']>0))]

In [16]:
Walmart_merged_clean_df['category'].replace({"Shop by Brand ": "Sports & Outdoors "}, inplace=True)

In [17]:
Walmart_merged_clean_df = Walmart_merged_clean_df[Walmart_merged_clean_df.product_name != "Mille Lacs Wild Rice Canoe Wild Rice, 15 oz"]
Walmart_merged_clean_df

,item_number,product_name,product_url,price_2019,price_2020,brand,category
0,554298081.0,Realtree Ladies Softshell Jacket,https://www.walmart.com/ip/Realtree-Ladies-Sof...,75.22,91.94,Mossy Oak,Sports & Outdoors
1,563412534.0,"Sechlers Hungarian Sweet-Mild Pepper Relish, 1...",https://www.walmart.com/ip/Sechler-s-Hungarian...,30.11,29.92,Sechlers,Food
2,554946697.0,Stealth Cam STCG34 G Series Trail Camera 12 MP...,https://www.walmart.com/ip/Stealth-Cam-STCG34-...,101.59,106.42,Stealth Cam,Sports & Outdoors
3,555023216.0,"Odor Eaters Stink Stoppers for Kids and Teens,...",https://www.walmart.com/ip/Odor-Eaters-Stink-S...,4.23,4.23,Odor-Eaters,Pets
4,560814213.0,Head Jr Alum Tennis Rckt 21In,https://www.walmart.com/ip/Head-Jr-Alum-Tennis...,18.55,18.55,Generic,Sports & Outdoors
...,...,...,...,...,...,...,...
419,550820800.0,"061012 Whistle Lanyard, Single, Red",https://www.walmart.com/ip/061012-Whistle-Lany...,11.99,11.99,Mendota,Sports & Outdoors
420,564332369.0,TRENDstarter Matte Finishing Wax,https://www.walmart.com/ip/TRENDstarter-Matte-...,9.63,9.63,TRENDstarter,Beauty
421,574378945.0,Venum Undisputed 2.0 MMA Gloves,https://www.walmart.com/ip/Venum-Undisputed-2-...,62.99,62.99,Venum,Sports & Outdoors
422,551857126.0,"Barska 1"" High Dovetail-Style Airgun .22 Rings",https://www.walmart.com/ip/Barska-1-High-Dovet...,7.35,9.00,Barska,Sports & Outdoors


In [18]:
Walmart_merged_clean_df['price_difference']=(Walmart_merged_clean_df['price_2020']-Walmart_merged_clean_df['price_2019'])
Walmart_merged_clean_df['price_percent_change']=(Walmart_merged_clean_df['price_difference']/Walmart_merged_clean_df['price_2019'])*100

In [19]:
Walmart_merged_clean_df = Walmart_merged_clean_df.round(2)

In [20]:
Walmart_merged_clean_df['category'] = Walmart_merged_clean_df['category'].str.rstrip(' ')
Walmart_merged_clean_df.head()

,item_number,product_name,product_url,price_2019,price_2020,brand,category,price_difference,price_percent_change
0,554298081.0,Realtree Ladies Softshell Jacket,https://www.walmart.com/ip/Realtree-Ladies-Sof...,75.22,91.94,Mossy Oak,Sports & Outdoors,16.72,22.23
1,563412534.0,"Sechlers Hungarian Sweet-Mild Pepper Relish, 1...",https://www.walmart.com/ip/Sechler-s-Hungarian...,30.11,29.92,Sechlers,Food,-0.19,-0.63
2,554946697.0,Stealth Cam STCG34 G Series Trail Camera 12 MP...,https://www.walmart.com/ip/Stealth-Cam-STCG34-...,101.59,106.42,Stealth Cam,Sports & Outdoors,4.83,4.75
3,555023216.0,"Odor Eaters Stink Stoppers for Kids and Teens,...",https://www.walmart.com/ip/Odor-Eaters-Stink-S...,4.23,4.23,Odor-Eaters,Pets,0.00,0.00
4,560814213.0,Head Jr Alum Tennis Rckt 21In,https://www.walmart.com/ip/Head-Jr-Alum-Tennis...,18.55,18.55,Generic,Sports & Outdoors,0.00,0.00


In [21]:
Walmart_merged_clean_df.to_sql(name = "walmartdata", con = engine, if_exists = "append", index = False)

In [22]:
pd.read_sql_query("select * from walmartdata", con = engine).head()

,item_number,product_name,product_url,price_2019,price_2020,price_difference,price_percent_change,brand,category
0,554298081.0,Realtree Ladies Softshell Jacket,https://www.walmart.com/ip/Realtree-Ladies-Sof...,75.22,91.94,16.72,22.23,Mossy Oak,Sports & Outdoors
1,563412534.0,"Sechlers Hungarian Sweet-Mild Pepper Relish, 1...",https://www.walmart.com/ip/Sechler-s-Hungarian...,30.11,29.92,-0.19,-0.63,Sechlers,Food
2,554946697.0,Stealth Cam STCG34 G Series Trail Camera 12 MP...,https://www.walmart.com/ip/Stealth-Cam-STCG34-...,101.59,106.42,4.83,4.75,Stealth Cam,Sports & Outdoors
3,555023216.0,"Odor Eaters Stink Stoppers for Kids and Teens,...",https://www.walmart.com/ip/Odor-Eaters-Stink-S...,4.23,4.23,0.00,0.00,Odor-Eaters,Pets
4,560814213.0,Head Jr Alum Tennis Rckt 21In,https://www.walmart.com/ip/Head-Jr-Alum-Tennis...,18.55,18.55,0.00,0.00,Generic,Sports & Outdoors


In [23]:
Walmart_merged_clean_df['price_percent_change'].max()

233.43